In [3]:
from os import listdir
from os.path import isfile, join
import string
import os

my_path=os.path.join('Bag_of_Words')
folders = [f for f in listdir(my_path)]
#print(folders)

files = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    files.append([f for f in listdir(folder_path)])

sum(len(files[i]) for i in range(5))

5000

In [4]:
pathname_list = []
for fo in range(len(folders)):
    for fi in files[fo]:
        pathname_list.append(join(my_path, join(folders[fo], fi)))
        
Y = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)
        
#print(Y)

In [5]:
#Split Data
from sklearn.model_selection import train_test_split
doc_train, doc_test, Y_train, Y_test = train_test_split(pathname_list, Y, random_state=0, test_size=0.3)

In [6]:
stopwords = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at',
 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 
 'can', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during',
 'each', 'few', 'for', 'from', 'further', 
 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's",
 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's",
 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself',
 "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself',
 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours' 'ourselves', 'out', 'over', 'own',
 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 
 'than', 'that',"that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", 
 "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 
 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
 "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's",'will', 'with', "won't", 'would', "wouldn't", 
 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 
 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'hundred', 'thousand', '1st', '2nd', '3rd',
 '4th', '5th', '6th', '7th', '8th', '9th', '10th']


In [7]:
#text pre=processing
def preprocess(words):
    
    table = str.maketrans('', '', '\t')
    words = [word.translate(table) for word in words]
    
    punctuations = (string.punctuation).replace("'", "") 
    trans_table = str.maketrans('', '', punctuations)
    stripped_words = [word.translate(trans_table) for word in words]
    words = [str for str in stripped_words if str]
    
    
    p_words = []
    for word in words:
        if (word[0] and word[len(word)-1] == "'"):
            word = word[1:len(word)-1]
        elif(word[0] == "'"):
            word = word[1:len(word)]
        else:
            word = word
        p_words.append(word)
    
    words = p_words.copy()
    words = [word for word in words if not word.isdigit()]
    words = [word for word in words if not len(word) == 1]
    words = [str for str in words if str]
    words = [word.lower() for word in words]
    words = [word for word in words if len(word) > 2]
    
    return words

def remove_stopwords(words):
    words = [word for word in words if not word in stopwords]
    return words

def tokenize_sentence(line):
    words = line[0:len(line)-1].strip().split(" ")
    words = preprocess(words)
    words = remove_stopwords(words)
    
    return words

def remove_metadata(lines):
    start=0
    for i in range(len(lines)):
        if(lines[i] == '\n'):
            start = i+1
            break
    new_lines = lines[start:]
    return new_lines

def tokenize(path):
    
    f = open(path, 'r')
    text_lines = f.readlines()
    text_lines = remove_metadata(text_lines)
    doc_words = []
    
    for line in text_lines:
        doc_words.append(tokenize_sentence(line))

    return doc_words

def flatten(list):
    new_list = []
    for i in list:
        for j in i:
            new_list.append(j)
    return new_list
len(folders)

5

In [8]:
import numpy as np

list_of_words = []
for document in doc_train:
    list_of_words.append(flatten(tokenize(document)))

print(len(list_of_words))
print(len(flatten(list_of_words)))
    
np_list_of_words = np.asarray(flatten(list_of_words))
words, counts = np.unique(np_list_of_words, return_counts=True)


freq, wrds = (list(i) for i in zip(*(sorted(zip(counts, words), reverse=True))))
#print(len(words))

3500
448961


In [9]:
#deciding the no. of words to use as feature

n = 5000
features = wrds[0:n]
#print(len(features))

dictionary = {}
doc_num = 1
print(len(list_of_words))
for doc_words in list_of_words:
    #print(doc_words)
    np_doc_words = np.asarray(doc_words)
    w, c = np.unique(np_doc_words, return_counts=True)
    dictionary[doc_num] = {}
    #print(len(w))
    for i in range(len(w)):
        dictionary[doc_num][w[i]] = c[i]
    doc_num = doc_num + 1   

3500


In [10]:
X_train = []
for k in dictionary.keys():
    row = []
    for f in features:
        if(f in dictionary[k].keys()):
            row.append(dictionary[k][f]) 
        else:
            row.append(0)
    X_train.append(row)
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

len(X_train)

3500

In [11]:
#FOR TEST DATA
list_of_words_test = []

for document in doc_test:
        list_of_words_test.append(flatten(tokenize(document)))

dictionary_test = {}
doc_num = 1
for doc_words in list_of_words_test:
    #print(doc_words)
    np_doc_words = np.asarray(doc_words)
    w, c = np.unique(np_doc_words, return_counts=True)
    dictionary_test[doc_num] = {}
    for i in range(len(w)):
        dictionary_test[doc_num][w[i]] = c[i]
    doc_num = doc_num + 1      

In [12]:
X_test = []
for k in dictionary_test.keys():
    row = []
    for f in features:
        if(f in dictionary_test[k].keys()):
            #if word f is present in the dictionary of the document as a key, its value is copied
            #this gives us no. of occurences
            row.append(dictionary_test[k][f]) 
        else:
            #if not present, the no. of occurences is zero
            row.append(0)
    X_test.append(row)

X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

print(len(X_test))

1500


In [13]:
# MULTINOMIAL NAIVE BAYES IMPLEMENTATION

def fit(X_train, Y_train):
    result = {}
    classes, counts = np.unique(Y_train, return_counts=True)
    
    for i in range(len(classes)):
        curr_class = classes[i]
        
        result["TOTAL_DATA"] = len(Y_train)
        result[curr_class] = {}
        
        X_tr_curr = X_train[Y_train == curr_class]
        
        #num_features = n
        num_features=n
        
        for j in range(num_features):
            result[curr_class][features[j]] = X_tr_curr[:,j].sum() 
                
        result[curr_class]["TOTAL_COUNT"] = counts[i]
    
    return result

def log_probablity(dictionary_train, x, curr_class):
    output = np.log(dictionary_train[curr_class]["TOTAL_COUNT"]) - np.log(dictionary_train["TOTAL_DATA"])
    num_words = len(x)
    for j in range(num_words):
        if(x[j] in dictionary_train[curr_class].keys()):
            xj = x[j]
            count_curr_class_equal_xj = dictionary_train[curr_class][xj] + 1
            count_curr_class = dictionary_train[curr_class]["TOTAL_COUNT"] + len(dictionary_train[curr_class].keys())
            curr_xj_prob = np.log(count_curr_class_equal_xj) - np.log(count_curr_class)
            output = output + curr_xj_prob
        else:
            continue
    
    return output

def predictSinglePoint(dictionary_train, x):
    classes = dictionary_train.keys()
    best_p = -10000
    best_class = -1
    for curr_class in classes:
        if(curr_class == "TOTAL_DATA"):
            continue
        p_curr_class = log_probablity(dictionary_train, x, curr_class)
        if(p_curr_class > best_p):
            best_p = p_curr_class
            best_class = curr_class
            
    return best_class

def predict(dictionary_train, X_test):
    Y_pred = []
    for x in X_test:
        y_predicted = predictSinglePoint(dictionary_train, x)
        Y_pred.append(y_predicted)
    
    #print(Y_pred)
    return Y_pred

In [14]:
train_dictionary = fit(X_train,Y_train)              #Fit data

In [15]:
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

X_test = []

for key in dictionary_test.keys():
    X_test.append(list(dictionary_test[key].keys()))

my_predictions = predict(train_dictionary, X_test)             #Predict data
my_predictions = np.asarray(my_predictions)

#metrics
print("Algorithm Accuracy:  ",accuracy_score(Y_test, my_predictions))

print(classification_report(Y_test, my_predictions))

Algorithm Accuracy:   0.858
              precision    recall  f1-score   support

  Automotive       0.99      0.73      0.84       284
    Graphics       0.91      0.95      0.93       299
    Politics       0.68      0.97      0.80       296
    Religion       0.84      0.82      0.83       297
      Sports       0.99      0.83      0.90       324

    accuracy                           0.86      1500
   macro avg       0.88      0.86      0.86      1500
weighted avg       0.88      0.86      0.86      1500

